In [2]:
from langchain_milvus import Milvus
from pymilvus.model.dense import JinaEmbeddingFunction

from src.config import Settings
from src.services.parser.factory import make_parser_service
from src.services.database.milvus_client import MilvusClient

In [4]:
settings = Settings()
parser_client = make_parser_service(settings)

2025-11-08 12:54:27,277 - INFO - 👌 Docling Parser Service initialized


In [3]:
filepath = 'assets/testpaper.pdf'
docs = await parser_client.parse_document_langchain(filepath)

2025-11-08 11:14:58,538 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-08 11:14:58,576 - INFO - Going to convert document batch...
2025-11-08 11:14:58,576 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 0f4e5e5da9415b1f1608f9dee1fe3e77
2025-11-08 11:14:58,584 - INFO - Loading plugin 'docling_defaults'
2025-11-08 11:14:58,586 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-11-08 11:14:58,586 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-11-08 11:14:58,808 - INFO - Accelerator device: 'mps'
2025-11-08 11:15:00,341 - INFO - Loading plugin 'docling_defaults'
2025-11-08 11:15:00,344 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-11-08 11:15:00,345 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-11-08 11:15:00

In [25]:
jina_embedding_function = JinaEmbeddingFunction(
    model_name=settings.jina.model_name, api_key=settings.jina.jina_api_key,
    task='retrieval.passage', dimensions=1024, late_chunking=True
        )
connection_args = {
    'uri': settings.milvus.uri, 'token': settings.milvus.api_key
}
store = Milvus(
    embedding_function=jina_embedding_function,
    connection_args=connection_args,
    collection_name=settings.milvus.collection_name,
    index_params={"index_type": "FLAT", "metric_type": "L2"},
    consistency_level="Strong",
    drop_old=True,
)

In [19]:
store.collection_description

''

In [5]:
vs = MilvusClient(settings)

HTTPError: Failed to create Milvus Vector Store: Too many custom fields: ['dense', 'sparse']. They cannot be mapped to a limited number of embedding functions, nor do they belong to any build-in function.

In [28]:
vs.vector_store

'PaperEmbedding'